## IMDB Dataset

In [ ]:
# Setup
!pip install fairseq

# Download the IMDB dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar zxvf aclImdb_v1.tar.gz

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [ ]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
# Format data
import os
import random
from glob import glob

def prepare_data(datadir):
    random.seed(0)
    for split in ['train', 'test']:
        samples = []
        for class_label in ['pos', 'neg']:
            fnames = glob(os.path.join(datadir, split, class_label) + '/*.txt')
            for fname in fnames:
                with open(fname, 'r') as fin:
                    line = fin.readline().strip()
                    samples.append((line, 1 if class_label == 'pos' else 0))
        random.shuffle(samples)
        out_fname = 'train' if split == 'train' else 'dev'
        with open(os.path.join(datadir, out_fname + '.input0'), 'w') as f1, \
             open(os.path.join(datadir, out_fname + '.label'), 'w') as f2:
            for sample in samples:
                f1.write(sample[0] + '\n')
                f2.write(str(sample[1]) + '\n')

prepare_data('aclImdb')

In [ ]:
!git clone https://github.com/pytorch/fairseq
!ls
# %cd fairseq

fatal: destination path 'fairseq' already exists and is not an empty directory.
aclImdb  aclImdb_v1.tar.gz  fairseq  sample_data


In [ ]:
!ls

aclImdb  aclImdb_v1.tar.gz  encoder.json  fairseq  sample_data	vocab.bpe


In [ ]:
%cd fairseq

/content/fairseq


In [ ]:
!ls

CODE_OF_CONDUCT.md  fairseq	   LICENSE	   RELEASE.md	     setup.py
CONTRIBUTING.md     fairseq_cli    MANIFEST.in	   release_utils.py  tests
docs		    hubconf.py	   pyproject.toml  scripts	     train.py
examples	    hydra_plugins  README.md	   setup.cfg


In [ ]:
# Download the BPE encoder and vocabulary
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json'
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe'

# BPE encoding of the data
!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "../aclImdb/train.input0" "../aclImdb/dev.input0" \
    --outputs "../aclImdb/train.input0.bpe" "../aclImdb/dev.input0.bpe" \
    --workers 60 \
    --keep-empty

--2024-05-04 20:52:39--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘encoder.json’

encoder.json        100%[===================>]   1018K  --.-KB/s    in 0.04s   

2024-05-04 20:52:39 (25.4 MB/s) - ‘encoder.json’ saved [1042301/1042301]

--2024-05-04 20:52:39--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K 

In [ ]:
# Download the dictionary for fairseq
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

# # Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.input0.bpe" \
    --validpref "../aclImdb/dev.input0.bpe" \
    --destdir "../IMDB-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.label" \
    --validpref "../aclImdb/dev.label" \
    --destdir "../IMDB-bin/label" \
    --workers 60


--2024-05-04 20:54:38--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.108, 3.163.189.96, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603290 (589K) [text/plain]
Saving to: ‘dict.txt’

dict.txt            100%[===================>] 589.15K  --.-KB/s    in 0.04s   

2024-05-04 20:54:38 (15.7 MB/s) - ‘dict.txt’ saved [603290/603290]

2024-05-04 20:54:40.501837: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 20:54:40.501903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 20:54:40.503600: E external/lo

In [ ]:
!pip install tensorboardX

!fairseq-train "/content/IMDB-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'imdb_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 1 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4

## Amazon Dataset


In [ ]:
# Step 1: Install the Hugging Face `datasets` library
!pip install datasets

# Import the necessary library
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("amazon_polarity")

# Accessing data
train_data = dataset["train"]
test_data = dataset["test"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})
Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 3600000
})


In [ ]:
# Print some sample data
for example in train_data.shuffle(seed=42).select(range(5)):
    print(f"Label: {example['label']}, Review Title: {example['title']}, Review Content: {example['content']}")

Label: 0, Review Title: Anyone who likes this better than the Pekinpah is a moron., Review Content: All the pretty people in this film. Even the Rudy character played by Michael Madsen. This is adapted from a Jim Thompson novel for cryin' out loud! These are supposed to be marginal characters, not fashion models. Though McQueen and McGraw were attractive (but check out McQueen's crummy prison haircut) they were believable in the role. Baldwin and Bassinger seem like movie stars trying to act like hard cases. Action wise, the robbery scene in the Pekinpah version was about 100 times more exciting and suspenseful than anything in this re-make.
Label: 0, Review Title: Author seems mentally unstable, Review Content: I know that Tom Robbins has a loyal following and I started the book with high expectations. However, I did not enjoy this book as it was too much work to follow his confused logic. I think that he was under the influence during most of time that he wrote.
Label: 1, Review Titl

In [ ]:
import pandas as pd

# Convert to pandas DataFrame
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

# Assuming binary classification where 'label' 0 and 1 are used
df_train['label'] = df_train['label'].replace({4: 1})
df_test['label'] = df_test['label'].replace({4: 1})

# Shuffle the DataFrame
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sample(frac=1, random_state=42)

df_train = df_train.head(25000)
df_test = df_test.head(25000)

print(df_train.head())

         label                                   title  \
2079998      0                          Expensive Junk   
1443106      0                          Toast too dark   
3463669      1   Excellent imagery...dumbed down story   
2914699      0  Are we pretending everyone is married?   
1603231      0                     Not worth your time   

                                                   content  
2079998  This product consists of a piece of thin flexi...  
1443106  Even on the lowest setting, the toast is too d...  
3463669  I enjoyed this disc. The video is stunning. I ...  
2914699  The authors pretend that parents neither die n...  
1603231  Might as well just use a knife, this product h...  


In [ ]:
!ls

CODE_OF_CONDUCT.md  encoder.json  hubconf.py	 pyproject.toml    scripts    train.py
CONTRIBUTING.md     examples	  hydra_plugins  README.md	   setup.cfg  vocab.bpe
dict.txt	    fairseq	  LICENSE	 RELEASE.md	   setup.py
docs		    fairseq_cli   MANIFEST.in	 release_utils.py  tests


In [ ]:
# !cd fairseq
# !ls

In [ ]:
df_train['content'].to_csv('train.input0', index=False, header=False)
df_train['label'].to_csv('train.label', index=False, header=False)
df_test['content'].to_csv('dev.input0', index=False, header=False)
df_test['label'].to_csv('dev.label', index=False, header=False)

# Check number of lines in each file
print("Training texts:", len(open('train.input0').readlines()))
print("Training labels:", len(open('train.label').readlines()))
print("Validation texts:", len(open('dev.input0').readlines()))
print("Validation labels:", len(open('dev.label').readlines()))

Training texts: 25000
Training labels: 25000
Validation texts: 25000
Validation labels: 25000


In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
# # BPE encoding of the data
!ls

!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "train.input0" \
    --outputs "train.input0.bpe" \
    --workers 60 \
    --keep-empty

!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "dev.input0" \
    --outputs "dev.input0.bpe" \
    --workers 60 \
    --keep-empty

# Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "train.input0.bpe" \
    --validpref "dev.input0.bpe" \
    --destdir "Amazon-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "train.label" \
    --validpref "dev.label" \
    --destdir "Amazon-bin/label" \
    --workers 60

CODE_OF_CONDUCT.md  docs	  hubconf.py	  README.md	    setup.py	  vocab.bpe
CONTRIBUTING.md     encoder.json  hydra_plugins   RELEASE.md	    tests
dev.input0	    examples	  LICENSE	  release_utils.py  train.input0
dev.label	    fairseq	  MANIFEST.in	  scripts	    train.label
dict.txt	    fairseq_cli   pyproject.toml  setup.cfg	    train.py
2024-05-04 21:02:13.459104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 21:02:13.459167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 21:02:13.461253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-04 21:02:13.4788

In [ ]:
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

--2024-05-04 21:26:50--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.108, 3.163.189.96, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘dict.txt’ not modified on server. Omitting download.



In [ ]:
!fairseq-train "Amazon-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'Amazon_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4

Streaming output truncated to the last 5000 lines.
epoch 003 | valid on 'valid' subset:  36% 567/1565 [00:31<00:57, 17.40it/s]
epoch 003 | valid on 'valid' subset:  36% 569/1565 [00:31<00:55, 17.80it/s]
epoch 003 | valid on 'valid' subset:  36% 571/1565 [00:31<00:57, 17.42it/s]
epoch 003 | valid on 'valid' subset:  37% 573/1565 [00:31<00:57, 17.25it/s]
epoch 003 | valid on 'valid' subset:  37% 575/1565 [00:32<00:57, 17.11it/s]
epoch 003 | valid on 'valid' subset:  37% 577/1565 [00:32<00:55, 17.81it/s]
epoch 003 | valid on 'valid' subset:  37% 579/1565 [00:32<00:55, 17.81it/s]
epoch 003 | valid on 'valid' subset:  37% 581/1565 [00:32<00:54, 18.01it/s]
epoch 003 | valid on 'valid' subset:  37% 583/1565 [00:32<00:57, 17.00it/s]
epoch 003 | valid on 'valid' subset:  37% 585/1565 [00:32<00:55, 17.59it/s]
epoch 003 | valid on 'valid' subset:  38% 587/1565 [00:32<00:57, 16.98it/s]
epoch 003 | valid on 'valid' subset:  38% 589/1565 [00:32<00:57, 17.10it/s]
epoch 003 | valid on 'valid' subset: 

In [ ]:
!ls

Amazon-bin	    dev.label	  fairseq_cli	  README.md	    tests
checkpoints	    dict.txt	  hubconf.py	  RELEASE.md	    train.input0
CODE_OF_CONDUCT.md  docs	  hydra_plugins   release_utils.py  train.input0.bpe
CONTRIBUTING.md     encoder.json  LICENSE	  scripts	    train.label
dev.input0	    examples	  MANIFEST.in	  setup.cfg	    train.py
dev.input0.bpe	    fairseq	  pyproject.toml  setup.py	    vocab.bpe
